<a href="https://colab.research.google.com/github/Stefi96/DetectingNFTs-Master/blob/main/Images_NFTs_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [16]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [13]:
# Function to load and preprocess images
def load_and_preprocess_images(image_paths, label):
    data = []
    labels = []
    for img_path in image_paths:
        try:
            img = Image.open(img_path).resize((128, 128))
            img_array = np.array(img) / 255.0
            data.append(img_array)
            labels.append(label)
        except Exception as e:
            pass
    return data, labels


In [22]:
# Paths to all the scam and legit images
# NOTE: Adjust these paths according to where you've stored the images in your Google Drive
cam_image_paths = [os.path.join("/content/drive/MyDrive/Master/Project/Images/Images_scam", image) for image in os.listdir("/content/drive/MyDrive/Master/Project/Images/Images_scam")]
egit_image_paths = [os.path.join("/content/drive/MyDrive/Master/Project/Images/Images_legit", image) for image in os.listdir("/content/drive/MyDrive/Master/Project/Images/Images_legit")]

In [26]:
import os

# Paths to your scam and legit images on Google Drive
scam_dir = "/content/drive/MyDrive/Master/Project/Images/Images_scam"
legit_dir = "/content/drive/MyDrive/Master/Project/Images/Images_legit"

# List the images in the directories
scam_images = os.listdir(scam_dir)
legit_images = os.listdir(legit_dir)

# Print the number of images detected in each directory
print(f"Number of scam images detected: {len(scam_images)}")
print(f"Number of legit images detected: {len(legit_images)}")

Number of scam images detected: 1082
Number of legit images detected: 1763


In [31]:
from PIL import Image
import numpy as np

def try_loading_images(image_paths):
    loaded_images = []
    for img_path in image_paths[:10]:  # Limiting to the first 10 images for testing
        try:
            img = Image.open(img_path).resize((128, 128))
            img_array = np.array(img) / 255.0
            loaded_images.append(img_array)
        except Exception as e:
            print(f"Error with image {img_path}: {e}")
    return loaded_images

# Attempt to load and process a subset of scam and legit images
loaded_scam_images = try_loading_images(scam_image_paths)
loaded_legit_images = try_loading_images(legit_image_paths)

print(f"Number of scam images loaded: {len(loaded_scam_images)}")
print(f"Number of legit images loaded: {len(loaded_legit_images)}")

Number of scam images loaded: 10
Number of legit images loaded: 10


In [29]:
scam_data, scam_labels = load_and_preprocess_images(scam_images, "scam")
legit_data, legit_labels = load_and_preprocess_images(legit_images, "legit")

all_data = scam_data + legit_data
all_labels = scam_labels + legit_labels

In [ ]:
# Loading all scam images
scam_data, scam_labels = load_and_preprocess_images(scam_image_paths, "scam")

# Loading all legit images
legit_data, legit_labels = load_and_preprocess_images(legit_image_paths, "legit")

# Printing the number of loaded images and labels
print(f"Number of scam images loaded: {len(scam_data)}")
print(f"Number of legit images loaded: {len(legit_data)}")

In [30]:
X_train, X_temp, y_train, y_temp = train_test_split(all_data, all_labels, test_size=0.3, random_state=42, stratify=all_labels)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

ValueError: ignored

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)


In [ ]:
y_train = to_categorical(y_train, 2)
y_val = to_categorical(y_val, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=32, validation_data=(np.array(X_val), np.array(y_val)))

In [ ]:
test_loss, test_acc = model.evaluate(np.array(X_test), np.array(y_test))
print(f"Test accuracy: {test_acc}")